<a href="https://colab.research.google.com/github/greeshmanth-5/Greeshmanth_INFO5731_Fall2023/blob/main/INFO5731_Assignment_Four_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA, LSA, and BERTopic. The following information should be reported:

(1) Features (text representation) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster.


In [ ]:
!pip install biterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for biterm: filename=biterm-0.1.5-cp310-cp310-linux_x86_64.whl size=235694 sha256=6877de3da5c96fc00d002cd5cc6f6c6a7866848676b713fd7c1edb1a3a385356
  Stored in directory: /root/.cache/pip/wheels/d9/6e/b8/b89e5a843baa692331b6c3db8d5f5ea43607ccad34426eebb4
Successfully built biterm


In [ ]:
# Write your code here

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import LdaModel
from biterm.utility import vec_to_biterms, topic_summuary
from sklearn.feature_extraction.text import TfidfVectorizer
from biterm.btm import oBTM
import numpy as np


file_path = '/content/imdb_reviews_annotated_1.csv'
imdb_data = pd.read_csv(file_path)

# Cleaning the text data
imdb_data['Cleaned Data'] = imdb_data['clean_text'].str.replace('[^\w\s]', '')  # Removal of punctuation
imdb_data['Cleaned Data'] = imdb_data['Cleaned Data'].apply(lambda x: " ".join(x.lower() for x in x.split()))  # Lowercase
nltk.download('stopwords')
stop = stopwords.words('english')
imdb_data['Cleaned Data'] = imdb_data['Cleaned Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# Tokenization and building the document-term matrix
tokenizer = RegexpTokenizer(r'\w+')
words = [tokenizer.tokenize(x) for x in pd.Series(imdb_data['Cleaned Data'])]
dictionary = corpora.Dictionary(words)
corpus = [dictionary.doc2bow(word) for word in words]

# LDA model
lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=20)
lda_model.print_topics(num_topics=10, num_words=5)

# Biterm Topic Model (oBTM)
biterm = imdb_data['Cleaned Data'].values
vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(biterm).toarray()  # Converting to tfidf matrix

text = np.array(vectorizer.get_feature_names_out())  # Getting the feature names
words = vec_to_biterms(X_tfidf)

# Fit oBTM model
btm_model = oBTM(num_topics=10, V=text)
btm_model.fit_transform(words, iterations=10)

# Getting the summary of topics
topic_summuary(btm_model.phi_wz.T, X_tfidf, text, 10)



<ipython-input-13-06eb250506a1>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  imdb_data['Cleaned Data'] = imdb_data['clean_text'].str.replace('[^\w\s]', '')  # Removal of punctuation
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 10/10 [09:24<00:00, 56.42s/it]


Topic 0 | Coherence=-19.01 | Top words= stephen film robbins best king dont hope great shawshank films
Topic 1 | Coherence=26.34 | Top words= visuals process maintaining whisper redeem witness tends manages seekthis response
Topic 2 | Coherence=-21.20 | Top words= freeman morgan robbins life hope great message acting prison performances
Topic 3 | Coherence=-4.16 | Top words= solace award day tim bob state stephen inmate story time
Topic 4 | Coherence=-20.63 | Top words= role office year classic box wonder prison comes nominations days
Topic 5 | Coherence=-34.09 | Top words= movie film time hope times believe really redemption greatest final
Topic 6 | Coherence=-19.44 | Top words= best great freeman story king robbins shawshank andy stephen acting
Topic 7 | Coherence=-37.25 | Top words= movie dont dufresne performance modern years time picture different far
Topic 8 | Coherence=-19.83 | Top words= movies make seven close day tough world think bob years
Topic 9 | Coherence=-31.40 | Top wo

{'coherence': [-19.009271313716898,
  26.339592861277897,
  -21.202301957140186,
  -4.162693941030649,
  -20.630099243732133,
  -34.09481260902916,
  -19.441245321980205,
  -37.254279312038335,
  -19.82711105633541,
  -31.398986505827935],
 'top_words': [array(['stephen', 'film', 'robbins', 'best', 'king', 'dont', 'hope',
         'great', 'shawshank', 'films'], dtype=object),
  array(['visuals', 'process', 'maintaining', 'whisper', 'redeem',
         'witness', 'tends', 'manages', 'seekthis', 'response'],
        dtype=object),
  array(['freeman', 'morgan', 'robbins', 'life', 'hope', 'great', 'message',
         'acting', 'prison', 'performances'], dtype=object),
  array(['solace', 'award', 'day', 'tim', 'bob', 'state', 'stephen',
         'inmate', 'story', 'time'], dtype=object),
  array(['role', 'office', 'year', 'classic', 'box', 'wonder', 'prison',
         'comes', 'nominations', 'days'], dtype=object),
  array(['movie', 'film', 'time', 'hope', 'times', 'believe', 'really',
    

In [ ]:
print(imdb_data.columns)

Index(['document_id', 'clean_text', 'sentiment'], dtype='object')


# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. Note: Cross-validation (5-fold or 10-fold) should be conducted. Here is the reference of cross-validation: https://scikit-learn.org/stable/modules/cross_validation.html.

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.

In [ ]:
"""I have added extra reviews which are negative and neutral in nature to perform this task as this
 task requires more than 1 value for the sentiment column and I have updated the csv file for this assignment."""

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Replace the file path with the correct path to your CSV file
file_path = '/content/imdb_reviews_annotated_1.csv'
df = pd.read_csv(file_path)

# Cleaning the text data
df['Cleaned Data'] = df['clean_text'].str.replace('[^\w\s]', '')  # Removal of punctuation
df['Cleaned Data'] = df['Cleaned Data'].apply(lambda x: " ".join(x.lower() for x in x.split()))  # Lowercase
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['Cleaned Data'] = df['Cleaned Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# Replace 'sentiment' with the correct column name for sentiment
sentiment_column = 'sentiment'

# Splitting data into 80-20 train and test data
tfidf_vector = TfidfVectorizer()
X_tfidf = tfidf_vector.fit_transform(df['Cleaned Data'])
x_train, x_test, y_train, y_test = train_test_split(X_tfidf,
                                                    df[sentiment_column],
                                                    test_size=0.2,
                                                    random_state=42)

# Checking the distribution of classes in the test set
print("Class distribution in the test set:")
print(y_test.value_counts())

# Multinomial Naive Bayes model
mnb = MultinomialNB()
model_mnb = mnb.fit(x_train, y_train)
y_pred_mnb = model_mnb.predict(x_test)
print('Accuracy of Multinomial Naive Bayes:', accuracy_score(y_pred_mnb, y_test))
print(classification_report(y_test, y_pred_mnb, zero_division=1))  # Setting zero_division=1 to handle zero divisions

# Cross-validation using MNB
scores_mnb = cross_val_score(mnb, x_test, y_test, cv=5)
print("Cross-validation using MNB:", scores_mnb.mean())

# Linear Support Vector Machine (SVM) model
svm = LinearSVC()
model_svm = svm.fit(x_train, y_train)
y_pred_svm = model_svm.predict(x_test)
print('Accuracy of SVM:', accuracy_score(y_pred_svm, y_test))
print(classification_report(y_test, y_pred_svm, zero_division=1))  # Setting zero_division=1 to handle zero divisions

# Cross-validation using SVM
# Use a smaller number of splits, e.g., cv=5
scores_svm = cross_val_score(svm, x_test, y_test, cv=5)
print("Cross-validation using SVM:", scores_svm.mean())


Class distribution in the test set:
Positive    5
Negative    2
Name: sentiment, dtype: int64
Accuracy of Multinomial Naive Bayes: 0.7142857142857143
              precision    recall  f1-score   support

    Negative       1.00      0.00      0.00         2
    Positive       0.71      1.00      0.83         5

    accuracy                           0.71         7
   macro avg       0.86      0.50      0.42         7
weighted avg       0.80      0.71      0.60         7

Cross-validation using MNB: 0.8
Accuracy of SVM: 0.7142857142857143
              precision    recall  f1-score   support

    Negative       1.00      0.00      0.00         2
    Positive       0.71      1.00      0.83         5

    accuracy                           0.71         7
   macro avg       0.86      0.50      0.42         7
weighted avg       0.80      0.71      0.60         7

Cross-validation using SVM: 0.8


<ipython-input-12-1fdeca43f0fb>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Cleaned Data'] = df['clean_text'].str.replace('[^\w\s]', '')  # Removal of punctuation
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


# **Question 3: House price prediction**

(20 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download from canvas. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Load the training and testing data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Combine training and testing data for preprocessing
combined_data = pd.concat([train_data.drop("SalePrice", axis=1), test_data])

# Separate numeric and categorical columns
numeric_cols = combined_data.select_dtypes(include=np.number).columns
categorical_cols = combined_data.select_dtypes(include='object').columns

# Impute missing values for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')
combined_data[numeric_cols] = numeric_imputer.fit_transform(combined_data[numeric_cols])

# Impute missing values for categorical columns
categorical_imputer = SimpleImputer(strategy='most_frequent')
combined_data[categorical_cols] = categorical_imputer.fit_transform(combined_data[categorical_cols])

# Converting categorical to numerical using label encoder
label_encoder = LabelEncoder()
for col in categorical_cols:
    combined_data[col] = label_encoder.fit_transform(combined_data[col])

# Splitting back into training and testing data
train_data_imputed = combined_data.iloc[:len(train_data)]
test_data_imputed = combined_data.iloc[len(train_data):]

# Scaling the data for better prediction
scaler = MinMaxScaler()
X = train_data_imputed
y = train_data["SalePrice"]

X_scaled_min_max = scaler.fit_transform(X)
X_scaled_min_max_df = pd.DataFrame(X_scaled_min_max, columns=X.columns)

X_scaled_min_max_test = scaler.transform(test_data_imputed)
X_scaled_min_max_df_test = pd.DataFrame(X_scaled_min_max_test, columns=test_data_imputed.columns)

# Splitting data into train and test (80-20)
x_train, x_test, y_train, y_test = train_test_split(X_scaled_min_max_df, y, test_size=0.2, random_state=0)

# Training the Linear Regression model
regressor = LinearRegression()
regressor.fit(x_train, y_train)

# Predicting the model
y_pred = regressor.predict(x_test)

# Model evaluation
print('Linear Regression R squared:', regressor.score(x_test, y_test))
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
print('Root Mean Squared Error:', rmse)

# Predicting house prices for the test data
predicted_prices = regressor.predict(X_scaled_min_max_df_test)

# Displaying results
test_data["Predicted_SalePrice"] = predicted_prices
test_data[['Id', 'Predicted_SalePrice']].to_csv('predicted_prices.csv', index=False)


Linear Regression R squared: 0.49830744666167914
Root Mean Squared Error: 58860.9675541636


# **Question 4: Using Pre-trained LLMs**

(20 points)
Utilize a **pre-trained Large Language Model (LLM) from the Hugging Face Repository** for your specific task using the data collected in Assignment 3. After creating an account on Hugging Face (https://huggingface.co/), choose a relevant LLM from their repository, such as GPT-3, BERT, or RoBERTa or any Meta based text analysis model. Provide a brief description of the selected LLM, including its original sources, significant parameters, and any task-specific fine-tuning if applied.

Perform a detailed analysis of the LLM's performance on your task, including key metrics, strengths, and limitations. Additionally, discuss any challenges encountered during the implementation and potential strategies for improvement. This will enable a comprehensive understanding of the chosen LLM's applicability and effectiveness for the given task.


In [ ]:


import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the IMDb Reviews dataset
df = pd.read_csv('/content/imdb_reviews_annotated - imdb_reviews_annotated (1).csv')

# Map string labels to numerical labels
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x.lower() == 'positive' else 0)

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define your dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = int(self.labels.iloc[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,  # Explicitly set truncation to True
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming 2 classes for sentiment

# Create datasets and data loaders
train_dataset = SentimentDataset(train_df['clean_text'], train_df['sentiment'], tokenizer, max_len=128)
test_dataset = SentimentDataset(test_df['clean_text'], test_df['sentiment'], tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Increased batch size
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Increased batch size

# Training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Using torch.optim.AdamW
epochs = 3

# Training loop
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation loop
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        true_labels.extend(labels.numpy())
        predicted_labels.extend(predictions.numpy())

accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Accuracy: 1.0


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Print confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(true_labels, predicted_labels)
print("Classification Report:")
print(class_report)


Confusion Matrix:
[[2 0]
 [0 5]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         5

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7



In [ ]:
import torch

# Function to tokenize and predict sentiment
def predict_sentiment(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        truncation=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    with torch.no_grad():
        model.eval()
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        output = model(input_ids, attention_mask=attention_mask)

    probs = torch.nn.functional.softmax(output.logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()

    return predicted_class, probs

# Take input from the user
user_input = input("Enter a movie review: ")

# Predict sentiment
predicted_class, class_probabilities = predict_sentiment(user_input)

# Display results
print("Predicted Class:", predicted_class)
print("Class Probabilities:", class_probabilities)


Enter a movie review: its an okay movie
Predicted Class: 1
Class Probabilities: tensor([[0.3912, 0.6088]])


For a dataset with columns like 'document_id', 'clean_text', and 'sentiment,' which involves sentiment analysis, we might consider using a pre-trained model specifically designed for natural language processing tasks. BERT (Bidirectional Encoder Representations from Transformers) and RoBERTa (Robustly optimized BERT approach) are popular choices for such tasks.

I have considered BERT for this analysis:

#### BERT (Bidirectional Encoder Representations from Transformers)

**Description:**
BERT, developed by Google, is a pre-trained transformer-based model for natural language processing tasks. Unlike traditional models that read text sequentially, BERT reads it bidirectionally, considering the entire context in a sentence. It has achieved state-of-the-art performance across various NLP tasks.

**Original Sources:**
BERT was introduced in the paper "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" by Devlin et al. The model is pretrained on a large corpus, including parts of BooksCorpus and English Wikipedia.

**Significant Parameters:**
- **Layers:** BERT consists of multiple layers (e.g., 12 layers in BERT base, 24 layers in BERT large).
- **Attention Heads:** Each layer has multiple attention heads (e.g., 12 heads per layer in BERT base).
- **Hidden Size:** The number of hidden units in each layer (e.g., 768 hidden units in BERT base).

**Fine-Tuning:**
For task-specific fine-tuning on sentiment analysis, you would typically add a classification layer on top of the BERT model and train it on your labeled sentiment data.

### Analysis of BERT Performance:

**Metrics:**
- **Accuracy:** Overall correctness of sentiment predictions.
- **Precision, Recall, F1 Score:** For each sentiment class, measuring the model's performance.
- **Confusion Matrix:** Understanding where the model makes errors.

**Strengths:**
- **Context Understanding:** BERT captures bidirectional context, allowing it to understand the nuanced meaning of words.
- **Transfer Learning:** Pre-trained on a massive dataset, BERT can be fine-tuned on smaller datasets for specific tasks.

**Limitations:**
- **Computational Intensity:** BERT is computationally expensive, both during training and inference.
- **Data Requirements:** It requires substantial labeled data for fine-tuning.
- **Token Limit:** BERT has a maximum token limit, and longer documents may need to be truncated.

**Challenges:**
- **Computational Resources:** Fine-tuning BERT may require significant computational resources.
- **Data Annotation:** Obtaining a labeled dataset for sentiment analysis is crucial.

**Strategies for Improvement:**
- **Model Size:** Depending on your computational resources, you may choose a smaller variant of BERT (e.g., DistilBERT).
- **Data Augmentation:** Augmenting your dataset with similar sentences to improve generalization.
- **Ensemble Models:** Combining predictions from multiple models can enhance performance.

Selecting the appropriate variant of BERT, managing computational resources, and addressing data-related challenges will be key to the successful implementation of sentiment analysis using a large language model like BERT.